In [0]:
#Graph Generation and Dijkstria
#import geopandas as gpd
import pandas as pd
import re
%matplotlib inline
import folium
from folium.plugins import MarkerCluster
import numpy as np
import random

In [0]:
result = pd.read_csv('boston_safe_lat&long_filtered.csv').drop(['Unnamed: 0'], axis = 1)

In [3]:
result.head()

,lat,long
0,42.306212,-71.136502
1,42.309115,-71.134452
2,42.311784,-71.132797
3,42.313186,-71.131863
4,42.314572,-71.130608


In [0]:
def haversine_np(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    miles = 6367 * c/1.609
    return miles

In [11]:
#Embassy Suites - 42.370535, -71.031580 (2966)
c = dict()
for i in range(result.shape[0]):
    c[str(result.index[i])] = haversine_np(42.370535, -71.031580, result.iloc[i][0], result.iloc[i][1])
cp = min(c, key=c.get)
cpdist = c[str(cp)]
cp

'2966'

In [12]:
#Maverick - 42.370040, -71.037984 (2999)
c = dict()
for i in range(result.shape[0]):
    c[str(result.index[i])] = haversine_np(42.370040, -71.037984, result.iloc[i][0], result.iloc[i][1])
cp = min(c, key=c.get)
cpdist = c[str(cp)]
cp

'2999'

In [13]:
#Bremen Porter - 42.372441, -71.034753 (4407)
c = dict()
for i in range(result.shape[0]):
    c[str(result.index[i])] = haversine_np(42.372441, -71.034753, result.iloc[i][0], result.iloc[i][1])
cp = min(c, key=c.get)
cpdist = c[str(cp)]
print(cp)

4407


In [14]:
#Cottage Maverick - 42.368331, -71.034753 (2969)
c = dict()
for i in range(result.shape[0]):
    c[str(result.index[i])] = haversine_np(42.368331, -71.034753, result.iloc[i][0], result.iloc[i][1])
cp = min(c, key=c.get)
cpdist = c[str(cp)]
print(cp)

2969


In [15]:
#Santarpio Pizzeria - 42.367117, -71.035709 (4210)
c = dict()
for i in range(result.shape[0]):
    c[str(result.index[i])] = haversine_np(42.372823, -71.035345, result.iloc[i][0], result.iloc[i][1])
cp = min(c, key=c.get)
cpdist = c[str(cp)]
print(cp)

4210


In [16]:
#Taco Mexo - 42.369998, -71.038767 (2997)
c = dict()
for i in range(result.shape[0]):
    c[str(result.index[i])] = haversine_np(42.369998, -71.038767, result.iloc[i][0], result.iloc[i][1])
cp = min(c, key=c.get)
cpdist = c[str(cp)]
print(cp)

2997


In [0]:
graph_list = [tuple(('Embassy Suites', 'Bremen Porter', haversine_np(42.370535, -71.031580, 42.372441, -71.034753 ))),
      tuple(('Embassy Suites', 'Santarpio Pizza', haversine_np(42.370535, -71.031580, 42.367117, -71.035709 ))),
      tuple(('Embassy Suites', 'Cottage Maverick', haversine_np(42.370535, -71.031580, 42.368331, -71.034753 ))),
      tuple(('Bremen Porter', 'Maverick Cafe', haversine_np(42.372441, -71.034753, 42.370040, -71.037984 ))),
      tuple(('Santarpio Pizza', 'Taco Mex', haversine_np(42.367117, -71.035709, 42.369998, -71.038767 ))),
      tuple(('Taco Mex', 'Maverick Cafe', haversine_np(42.369998, -71.038767, 42.370040, -71.037984 ))),
      tuple(('Cottage Maverick', 'Maverick Cafe', haversine_np(42.368331, -71.034753, 42.370040, -71.037984 )))]

In [18]:
graph_list

[('Embassy Suites', 'Bremen Porter', 0.2232798672741459),
 ('Embassy Suites', 'Santarpio Pizza', 0.29530898103753117),
 ('Embassy Suites', 'Cottage Maverick', 0.22465763258142837),
 ('Bremen Porter', 'Maverick Cafe', 0.22956248279508543),
 ('Santarpio Pizza', 'Taco Mex', 0.22087556293653007),
 ('Taco Mex', 'Maverick Cafe', 0.05408587892833907),
 ('Cottage Maverick', 'Maverick Cafe', 0.2264205815462706)]

In [0]:
#Final Dijkstria

from collections import deque, namedtuple


# we'll use infinity as a default distance to nodes.
inf = float('inf')
Edge = namedtuple('Edge', 'start, end, cost')


def make_edge(start, end, cost=1):
  return Edge(start, end, cost)


class Graph:
    def __init__(self, edges):
        # let's check that the data is right
        wrong_edges = [i for i in edges if len(i) not in [2, 3]]
        if wrong_edges:
            raise ValueError('Wrong edges data: {}'.format(wrong_edges))

        self.edges = [make_edge(*edge) for edge in edges]

    @property
    def vertices(self):
        return set(
            sum(
                ([edge.start, edge.end] for edge in self.edges), []
            )
        )

    def get_node_pairs(self, n1, n2, both_ends=True):
        if both_ends:
            node_pairs = [[n1, n2], [n2, n1]]
        else:
            node_pairs = [[n1, n2]]
        return node_pairs

    def remove_edge(self, n1, n2, both_ends=True):
        node_pairs = self.get_node_pairs(n1, n2, both_ends)
        edges = self.edges[:]
        for edge in edges:
            if [edge.start, edge.end] in node_pairs:
                self.edges.remove(edge)

    def add_edge(self, n1, n2, cost=1, both_ends=True):
        node_pairs = self.get_node_pairs(n1, n2, both_ends)
        for edge in self.edges:
            if [edge.start, edge.end] in node_pairs:
                return ValueError('Edge {} {} already exists'.format(n1, n2))

        self.edges.append(Edge(start=n1, end=n2, cost=cost))
        if both_ends:
            self.edges.append(Edge(start=n2, end=n1, cost=cost))

    @property
    def neighbours(self):
        neighbours = {vertex: set() for vertex in self.vertices}
        for edge in self.edges:
            neighbours[edge.start].add((edge.end, edge.cost))

        return neighbours

    def dijkstra(self, source, dest):
        assert source in self.vertices, 'Such source node doesn\'t exist'
        distances = {vertex: inf for vertex in self.vertices}
        previous_vertices = {
            vertex: None for vertex in self.vertices
        }
        distances[source] = 0
        vertices = self.vertices.copy()

        while vertices:
            current_vertex = min(
                vertices, key=lambda vertex: distances[vertex])
            vertices.remove(current_vertex)
            if distances[current_vertex] == inf:
                break
            for neighbour, cost in self.neighbours[current_vertex]:
                alternative_route = distances[current_vertex] + cost
                if alternative_route < distances[neighbour]:
                    distances[neighbour] = alternative_route
                    previous_vertices[neighbour] = current_vertex

        path, current_vertex = deque(), dest
        while previous_vertices[current_vertex] is not None:
            path.appendleft(current_vertex)
            current_vertex = previous_vertices[current_vertex]
        if path:
            path.appendleft(current_vertex)
        return path



In [25]:
graph = Graph(graph_list)
print(graph.dijkstra("Embassy Suites", "Maverick Cafe"))

deque(['Embassy Suites', 'Cottage Maverick', 'Maverick Cafe'])


In [0]:
####### failed attempts at graph generation from here on
####### Just for reference
####### DO NOT USE

In [0]:
#final final code
from collections import Counter 

graph_list = []
start = sample.iloc[[random.choice(list(range(sample.shape[0])))]]
others = sample.drop(start.index[0])

while len(graph_list) < 100:

  dist_start_others = dict()
  for i in range(others.shape[0]):
      dist_start_others[str(others.index[i])] = haversine_np(start.iloc[0][0], start.iloc[0][1], others.iloc[i][0], others.iloc[i][1])

  p11 = min(dist_start_others, key=dist_start_others.get)
  p11dist = dist_start_others[str(p11)]
  layer_1 = dict()
  layer_1[int(p11)] = p11dist

  del dist_start_others[str(p11)]

  p12 = min(dist_start_others, key=dist_start_others.get)
  p12dist = dist_start_others[str(p12)]
  layer_1[int(p12)] = p12dist


  graph_list.append(tuple((str(start.index[0]), p11,  p11dist )))
  graph_list.append(tuple((str(start.index[0]), p12,  p12dist )))


  #layer11
  to_drop = []
  for j in range(len(list(layer_1.keys()))):
    to_drop.append(int(list(layer_1.keys())[j]))
  others_11 = others.drop(to_drop)
  node_11 = sample.iloc[[int(p11)]]
  dist_11_2 = dict()
  for i in range(others_11.shape[0]):
      dist_11_2[str(others_11.index[i])] = haversine_np(node_11.iloc[0][0], node_11.iloc[0][1], others_11.iloc[i][0], others_11.iloc[i][1])

  p21 = min(dist_11_2, key=dist_11_2.get)
  p21dist = dist_11_2[str(p21)]
  layer_2u = dict()
  layer_2u[int(p21)] = p21dist

  del dist_11_2[str(p21)]

  p22 = min(dist_11_2, key=dist_11_2.get)
  p22dist = dist_11_2[str(p22)]
  layer_2u[int(p22)] = p22dist


  graph_list.append(tuple((str(node_11.index[0]), p21,  p21dist )))
  graph_list.append(tuple((str(node_11.index[0]), p22,  p22dist )))



  #layer12
  to_drop = []
  for j in range(len(list(layer_2u.keys()))):
    to_drop.append(int(list(layer_2u.keys())[j]))
  others_12 = others_11.drop(to_drop)
  node_12 = sample.iloc[[int(p12)]]
  dist_12_2 = dict()
  for i in range(others_12.shape[0]):
      dist_12_2[str(others_12.index[i])] = haversine_np(node_12.iloc[0][0], node_12.iloc[0][1], others_12.iloc[i][0], others_12.iloc[i][1])

  p23 = min(dist_12_2, key=dist_12_2.get)
  p23dist = dist_12_2[str(p23)]
  layer_2l = dict()
  layer_2l[int(p23)] = p23dist

  del dist_12_2[str(p23)]

  p24 = min(dist_12_2, key=dist_12_2.get)
  p24dist = dist_12_2[str(p24)]
  layer_2l[int(p24)] = p24dist


  graph_list.append(tuple((str(node_12.index[0]), p23,  p23dist )))
  graph_list.append(tuple((str(node_12.index[0]), p24,  p24dist )))


  #Layer-2
  to_drop = []
  for j in range(len(list(layer_2l.keys()))):
    to_drop.append(int(list(layer_2l.keys())[j]))
  others_2 = others_12.drop(to_drop)
  node_21 = sample.iloc[[int(p21)]]
  node_22 = sample.iloc[[int(p22)]]
  node_23 = sample.iloc[[int(p23)]]
  node_24 = sample.iloc[[int(p24)]]

  dist_2_3 = dict()
  for i in range(others_2.shape[0]):
      dist_2_3[str(others_2.index[i])] = haversine_np(node_21.iloc[0][0], node_21.iloc[0][1], others_2.iloc[i][0], others_2.iloc[i][1]) + haversine_np(node_22.iloc[0][0], node_22.iloc[0][1], others_2.iloc[i][0], others_2.iloc[i][1]) + haversine_np(node_23.iloc[0][0], node_23.iloc[0][1], others_2.iloc[i][0], others_2.iloc[i][1]) + haversine_np(node_24.iloc[0][0], node_24.iloc[0][1], others_2.iloc[i][0], others_2.iloc[i][1])

  p3 = min(dist_2_3, key=dist_2_3.get)
  p3dist = dist_2_3[str(p3)]
  layer_3 = dict()
  layer_3[int(p3)] = p3dist

  graph_list.append(tuple((str(node_21.index[0]), p3,  haversine_np(node_21.iloc[0][0], node_21.iloc[0][1], others_2.iloc[int(p3)][0], others_2.iloc[int(p3)][1]) )))
  graph_list.append(tuple((str(node_22.index[0]), p3,  haversine_np(node_22.iloc[0][0], node_22.iloc[0][1], others_2.iloc[int(p3)][0], others_2.iloc[int(p3)][1]) )))
  graph_list.append(tuple((str(node_23.index[0]), p3,  haversine_np(node_23.iloc[0][0], node_23.iloc[0][1], others_2.iloc[int(p3)][0], others_2.iloc[int(p3)][1]) )))
  graph_list.append(tuple((str(node_24.index[0]), p3,  haversine_np(node_24.iloc[0][0], node_24.iloc[0][1], others_2.iloc[int(p3)][0], others_2.iloc[int(p3)][1]) )))

  others_3 = others_2.drop(int(p3))

  start = sample.iloc[[int(p3)]]
  others = others_3
  print(len(graph_list))

IndexError: ignored

In [0]:
start = sample.iloc[[0]]
others = sample.drop(start.index[0])
graph_list = []

dist_start_others = dict()
for i in range(others.shape[0]):
    dist_start_others[str(others.index[i])] = haversine_np(start.iloc[0][0], start.iloc[0][1], others.iloc[i][0], others.iloc[i][1])

p11 = min(dist_start_others, key=dist_start_others.get)
p11dist = dist_start_others[str(p11)]
layer_1 = dict()
layer_1[int(p11)] = p11dist

del dist_start_others[str(p11)]

p12 = min(dist_start_others, key=dist_start_others.get)
p12dist = dist_start_others[str(p12)]
layer_1[int(p12)] = p12dist

graph_list.append(tuple((str(start.index[0]), p11,  p11dist )))
graph_list.append(tuple((str(start.index[0]), p12,  p12dist )))

graph_list.append(tuple(('9924', '10000', 0.5)))
graph_list.append(tuple(('9923', '10000', 0.5)))


In [0]:
dist_dict = dict()
graphlist2 = []
start = sample.iloc[[0]]
#ct = 1
others = sample.drop(start.index[0])

for i in range(others):
  


others = sample.iloc[random.sample(list(range(sample.shape[0])), k = 3)]
for j in range(others.shape[0]):
  to_compare = others.iloc[[j]]
  dist = haversine_np(start.iloc[0][0], start.iloc[0][1], to_compare.iloc[0][0], to_compare.iloc[0][1])
  graphlist2.append(tuple((str(start.index[0]), str(to_compare.index[0]),  dist )))

for k in range(others.shape[0]):
  start = others.iloc[[k]]

